In [1]:
import numpy as np
import scipy as sc
import random
from tqdm.notebook import tqdm
from scipy.optimize import minimize
from scipy import linalg
from scipy.sparse import csr_matrix
from scipy.special import expit as sigmoid
from scipy.special import log_expit
from joblib import Parallel, delayed

In [2]:
import os
import shutil
import sys

import numpy as np
from scipy import sparse

import matplotlib.pyplot as plt
%matplotlib inline
#
import seaborn as sn
#sn.set()

import pandas as pd

import bottleneck as bn

import datetime
from copy import deepcopy

In [3]:

import pickle

def save_pkl(obj, filename ):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL )
    
def load_pkl(filename ):
    with open(filename, 'rb') as f:
        return pickle.load(f)
 

In [4]:
DATA_DIR = '/efs/users/hsteck/public/data_for_ease/movielens20mio/'

In [5]:
pro_dir = os.path.join(DATA_DIR, 'pro_sg')


In [6]:
unique_sid = list()
with open(os.path.join(pro_dir, 'unique_sid.txt'), 'r') as f:
    for line in f:
        unique_sid.append(line.strip())

n_items = len(unique_sid)

def load_train_data(csv_file):
    tp = pd.read_csv(csv_file)
    n_users = tp['uid'].max() + 1

    rows, cols = tp['uid'], tp['sid']
    data = sparse.csr_matrix((np.ones_like(rows),
                             (rows, cols)), dtype='float64',
                             shape=(n_users, n_items))
    return data

In [7]:
random.sample(range(10),3)

[6, 1, 7]

In [8]:
def load_xtx_binary():
    #items = random.sample(range(20108),1000)
    train_data = load_train_data(os.path.join(pro_dir, 'train.csv'))
    X=train_data
    print (X.shape)
    return [X.shape[0] , X]

In [9]:
userCnt,X=load_xtx_binary()
I = 20108
U = 116677
W = np.ones((U,I))
W[X.nonzero()] = 40.0

(116677, 20108)


In [24]:
def ease(X, lam = 200.0):
    print('multiplying matrix')
    G = X.T @ X
    diagIndices = np.diag_indices(G.shape[0])
    G = G + lam * np.eye(G.shape[0])
    print('inverting')
    P = linalg.inv(G)
    print('inverting complete')
    B = P / (-np.diag(P))
    B[diagIndices] = 0.0
    return B

def ease_ones(X, lam = 200.0):
    print('multiplying matrix')
    ones = np.ones((U,I))
    G = X.T @ X + X.T@ones + ones.T@X + 1
    diagIndices = np.diag_indices(G.shape[0])
    G = G + lam * np.eye(G.shape[0])
    print('inverting')
    P = linalg.inv(G)
    print('inverting complete')
    B = P / (-np.diag(P))
    B[diagIndices] = 0.0
    return B

In [12]:
W_lin = ease(X,lam=100.0)

multiplying matrix
inverting
inverting complete


In [25]:
W_ones = ease_ones(X,lam=100.0)

multiplying matrix
inverting
inverting complete


In [20]:
evaluate(W_lin)

2024-09-05 20:06:48.320954
0 ... 5000
5000 ... 10000
Test NDCG@100=0.41643 (0.00215)
Test Recall@20=0.38927 (0.00267)
Test Recall@50=0.51623 (0.00282)
2024-09-05 20:07:00.232360


[0.41643417019034085, 0.389274373357121, 0.5162321377128388]

In [26]:
evaluate(W_ones)

2024-09-05 20:12:09.659813
0 ... 5000
5000 ... 10000
Test NDCG@100=0.41657 (0.00215)
Test Recall@20=0.38929 (0.00267)
Test Recall@50=0.51625 (0.00282)
2024-09-05 20:12:21.684234


[0.41657232043263487, 0.3892876778918536, 0.5162468991208307]

In [14]:
def load_tr_te_data(csv_file_tr, csv_file_te):
    tp_tr = pd.read_csv(csv_file_tr)
    tp_te = pd.read_csv(csv_file_te)

    start_idx = min(tp_tr['uid'].min(), tp_te['uid'].min())
    end_idx = max(tp_tr['uid'].max(), tp_te['uid'].max())

    rows_tr, cols_tr = tp_tr['uid'] - start_idx, tp_tr['sid']
    rows_te, cols_te = tp_te['uid'] - start_idx, tp_te['sid']

    data_tr = sparse.csr_matrix((np.ones_like(rows_tr),
                             (rows_tr, cols_tr)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    data_te = sparse.csr_matrix((np.ones_like(rows_te),
                             (rows_te, cols_te)), dtype='float64', shape=(end_idx - start_idx + 1, n_items))
    return data_tr, data_te

In [15]:
test_data_tr, test_data_te = load_tr_te_data(
    os.path.join(pro_dir, 'test_tr.csv'),
    os.path.join(pro_dir, 'test_te.csv'))

In [16]:
N_test = test_data_tr.shape[0]
idxlist_test = range(N_test)


In [17]:
def NDCG_binary_at_k_batch(X_pred, heldout_batch, k=100):
    '''
    normalized discounted cumulative gain@k for binary relevance
    ASSUMPTIONS: all the 0's in heldout_data indicate 0 relevance
    '''
    batch_users = X_pred.shape[0]
    idx_topk_part = bn.argpartition(-X_pred, k, axis=1)
    topk_part = X_pred[np.arange(batch_users)[:, np.newaxis],
                       idx_topk_part[:, :k]]
    idx_part = np.argsort(-topk_part, axis=1)
    # X_pred[np.arange(batch_users)[:, np.newaxis], idx_topk] is the sorted
    # topk predicted score
    idx_topk = idx_topk_part[np.arange(batch_users)[:, np.newaxis], idx_part]
    # build the discount template
    tp = 1. / np.log2(np.arange(2, k + 2))

    DCG = (heldout_batch[np.arange(batch_users)[:, np.newaxis],
                         idx_topk].toarray() * tp).sum(axis=1)
    IDCG = np.array([(tp[:min(n, k)]).sum()
                     for n in heldout_batch.getnnz(axis=1)])
    return DCG / IDCG

In [18]:
def Recall_at_k_batch(X_pred, heldout_batch, k=100):
    batch_users = X_pred.shape[0]

    idx = bn.argpartition(-X_pred, k, axis=1)
    X_pred_binary = np.zeros_like(X_pred, dtype=bool)
    X_pred_binary[np.arange(batch_users)[:, np.newaxis], idx[:, :k]] = True

    X_true_binary = (heldout_batch > 0).toarray()
    tmp = (np.logical_and(X_true_binary, X_pred_binary).sum(axis=1)).astype(
        np.float32)
    recall = tmp / np.minimum(k, X_true_binary.sum(axis=1))
    return recall

In [19]:
def evaluate(BB):
    #evaluate in batches
    print(datetime.datetime.now())

    #makeSparseFormat(BB, 0.0)


    batch_size_test=5000
    n100_list, r20_list, r50_list = [], [], []



    for bnum, st_idx in enumerate(range(0, N_test, batch_size_test)):
        end_idx = min(st_idx + batch_size_test, N_test)
        Xtest = test_data_tr[idxlist_test[st_idx:end_idx]]
        #Xtest = Xtest[:,-I:]
        print (str(st_idx)+' ... '+str(end_idx))
        if sparse.isspmatrix(Xtest):
            Xtest = Xtest.toarray()
        Xtest = Xtest.astype('float32')

        #pred_val = Xtest.dot(BB_excl)
        #pred_val = (((Xtest-mu) * scaling).dot(BBth) / scaling) +mu   # no bias
        #pred_val = Xtest.dot(beta_0d)  # no bias
        #pred_val =Xtest.dot(beta_lowrank)  
        pred_val = Xtest.dot(BB)

        # exclude examples from training and validation (if any)
        pred_val[Xtest.nonzero()] = -np.inf
        n100_list.append(NDCG_binary_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=100))
        r20_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=20))
        r50_list.append(Recall_at_k_batch(pred_val, test_data_te[idxlist_test[st_idx:end_idx]], k=50))
        #calc_coverageCounts(coverageCounts2, pred_val)
        #break  # do only 5000 users

    n100_list = np.concatenate(n100_list)
    r20_list = np.concatenate(r20_list)
    r50_list = np.concatenate(r50_list)

    print("Test NDCG@100=%.5f (%.5f)" % (np.mean(n100_list), np.std(n100_list) / np.sqrt(len(n100_list))))
    print("Test Recall@20=%.5f (%.5f)" % (np.mean(r20_list), np.std(r20_list) / np.sqrt(len(r20_list))))
    print("Test Recall@50=%.5f (%.5f)" % (np.mean(r50_list), np.std(r50_list) / np.sqrt(len(r50_list))))

    print(datetime.datetime.now())
    return [np.mean(n100_list), np.mean(r20_list), np.mean(r50_list)]

In [ ]:
# #Bn_ = (np.ones((I , I)) - 3).flatten()
# Bn = (np.zeros((I , I))).flatten()
# Bn1 = (np.ones((I , I)) + 2).flatten()
# for i in range(10000):
#     #Bnvec = Bn.flatten()
#     #Bn_vec = Bn_.flatten()
#     dBn_ = der(Bn_).flatten()
#     dBn = der(Bn).flatten()
#     top = np.abs(np.inner(Bn - Bn_, dBn - dBn_))
#     bottom = np.linalg.norm(dBn - dBn_) ** 2
#     if bottom <= 1e-132:
#         break
#     gamma = top / bottom
#     #print(gamma)
#     print(np.linalg.norm(der(Bn)))
#     Bn1 = Bn - gamma * der(Bn)
#     Bn_ = Bn
#     Bn = Bn1

# def temp(W):
#     W = W.reshape(I,I)
#     sums = 0.0
#     for i in range(I):
#         xw = X@W[:,i]
#         sums += np.sum( np.logaddexp(-xw/2,xw/2) - X[:,i]/2 * xw)
#     return sums



# def temp2(W, lam = 1.0):
#     W = W.reshape(I,I)
#     S = X @ W
#     return np.sum(np.logaddexp(-S/2,S/2) - X / 2 * S) + lam * np.linalg.norm(W,ord='fro') ** 2


# def der2(W, lam = 1.0):
#     W = W.reshape(I,I)
#     S = X @ W
#     return (1/2 * X.T @ (sigmoid(S/2) - (sigmoid(-S/2)) + X) + 2 * lam * W).flatten()
    



# def der(W):
#     W = W.reshape(I,I)
#     der = np.zeros((I,I))
#     for u in range(U):
#         for i in range(I):
#             scalar = 1 / 2 * (np.tanh(np.inner(X[u],W[:,i])) - X[u,i])
#             der[i] = der[i] + scalar * X[u]
#     return der.flatten()

# def Hess(W):
#     hess = np.zeros((I,I))
#     for u in range(U):
#         for i in range(I):
#             scalar = 1 / (2 * np.cosh(np.inner(X[u],W[:,i])) + 2)
#             hess = hess + scalar * np.outer(X[u],X[u])
#     return hess

# def lsq(B, X, lam = 1.0):
#     B = B.reshape(I,I)
#     S = X @ B
#     loss = sc.linalg.norm(X - S, ord='fro') ** 2 + lam * (sc.linalg.norm(B, ord='fro') ** 2)
#     return loss

# def dlsq(B, X, lam = 1.0):
#     B = B.reshape(I,I)
#     S = X @ B
#     return (2 * lam * B - 2 * X.T @ (X - S)).flatten()


# def llog(Bvec, X, lam = 1.0):
#     B = Bvec.reshape(I,I)
#     S = X @ B
#     Svec = S.flatten()
#     Xvec = X.flatten()

#     loss = np.sum(np.logaddexp(-Svec/2,Svec/2) - Xvec/2*Svec)
    
#     loss = lam * np.inner(Bvec,Bvec) + loss
#     return loss


# def llog2(Bvec, X, lam = 1.0):
#     B = Bvec.reshape(I,I)
#     S = X @ B
#     Svec = S.flatten()
#     Xvec = X.flatten()

#     loss = np.sum(np.logaddexp(-S/2,S/2) - X/2*S)
    
#     loss = lam * np.inner(Bvec,Bvec) + loss
#     return loss

# def dllog2(Bvec, X, lam = 1.0):
#     B = Bvec.reshape(I,I)
#     S = X @ B
#     return (1/2 * X.T @ (2*sigmoid(S) - np.ones((U,I)) + X) + 2 * lam * B).flatten
    
    
     

In [ ]:
# Bn = np.zeros(I * I)  
# Bn_ = np.zeros(I * I) + 10 
# Bn1 = np.zeros(I * I) + 100
# gamma = 1.0
# lam = 0.0
# while gamma > 1e-10:
#     dBn_ = der2(Bn_)
#     dBn = der2(Bn)
#     top = np.abs(np.inner(Bn - Bn_, dBn - dBn_))
#     bottom = np.linalg.norm(dBn - dBn_) ** 2
#     if bottom <= 1e-32:
#         break
#     gamma = top / bottom
#     print(temp2(Bn1),np.linalg.norm(der2(Bn1)),gamma)
#     Bn1 = Bn - gamma * der2(Bn)
#     #Bn1mat = Bn1.reshape(I,I)
#     #Bn1 = (Bn1mat - np.diag(np.diag(Bn1mat))).flatten()
#     Bn_ = Bn
#     Bn = Bn1

In [ ]:
# print(temp2(Bn1),np.linalg.norm(der2(np.eye(I) * 100)))

In [ ]:
# W = ease(X)

In [ ]:
# np.linalg.norm(W - Bn1.reshape(I,I))

In [ ]:
# U = 200_000
# I = 20_000
# lam = 100
# #X = np.zeros((U,I))

# p = np.random.uniform(size=I)
# print('sampling matrix')
# X_ = np.random.binomial(n=1,p=p,size=(U,I)) 
# X = csr_matrix(X_)
# X_ = []

In [ ]:
mask = np.ones(I, dtype=bool)
mask[0] = False
w = np.zeros(I-1)
item = 0

In [ ]:
hess = (np.diag(Var[:,item]**2)@X[:,mask]).T @ X[:,mask]
print('1')
diff=(X[:,mask]@w).reshape(U,1) - X[:,item]
print('2')
grad = X[:,mask].T @ (Var[:,item]**2*diff).T

In [ ]:
w = w-sc.sparse.linalg.spsolve(hess,grad)

In [ ]:
w

In [ ]:
np.multiply(np.multiply(Var[:,item],diff),Var[:,item])

In [ ]:
hess.toar

In [ ]:
X.T.shape